In [1]:

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import math
import torchvision.transforms as transforms

# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


def train_test_split_custom(X, y=None, test_size=0.1, random_state=None, stratify=None):
    if random_state is not None:
        torch.manual_seed(random_state)
    if isinstance(test_size, float):
        test_size = int(test_size * len(X))
    indices = torch.randperm(len(X))
    if stratify is not None:
        stratify_indices = torch.argsort(torch.Tensor(stratify))
        indices = indices[stratify_indices]

    test_indices = indices[:test_size]
    train_indices = indices[test_size:]

    X_train, X_test = X[train_indices], X[test_indices]
    if not y:
        return X_train, X_test
    else :
        y_train, y_test = y[train_indices], y[test_indices]

    return X_train, X_test, y_train, y_test

# Image Downloader
image_dataset_downloader = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform)

image_dataset_downloader_test = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform)

train_idx, validation_idx = train_test_split_custom(torch.arange(len(image_dataset_downloader)),
                                             test_size=0.1,
                                             random_state=999,
                                             stratify=image_dataset_downloader.targets)

image_train_data = torch.utils.data.Subset(image_dataset_downloader,train_idx)
image_val_data = torch.utils.data.Subset(image_dataset_downloader,validation_idx)

class ImageDataset(Dataset):
    def __init__(self, split):
        super().__init__()
        self.datasplit = split

        if self.datasplit == "train":
            self.dataset = image_train_data
        elif self.datasplit == "val":
            self.dataset = image_val_data
        elif self.datasplit == "test":
            self.dataset = image_dataset_downloader_test
        else:
            raise ValueError("Invalid split argument. Choose from 'train', 'val', or 'test'.")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img, target = self.dataset[index]
        return img, target


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        shortcut = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += shortcut
        out = self.relu(out)
        return out



class Resnet_Q1(nn.Module):
    def __init__(self,): # DO NOT CHANGE __init__ arguements
        super(Resnet_Q1, self).__init__()
        self.initial_conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.initial_bn1 = nn.BatchNorm2d(64)
        self.initial_relu1 = nn.ReLU(inplace=True)
        self.initial_conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.initial_bn2 = nn.BatchNorm2d(64)
        self.initial_relu2 = nn.ReLU(inplace=True)

        # Define rest 17 Residual Blocks
        self.layer1 = self._make_layer(17, 64, 64)

        # Final  full connected layer
        self.fc = nn.Linear(65536, 10)

    def _make_layer(self, num_blocks, in_channels, out_channels):
        layers = []
        for _ in range(num_blocks):
            layers.append(ResidualBlock(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial_conv1(x)
        x = self.initial_bn1(x)
        x = self.initial_relu1(x)
        x = self.initial_conv2(x)
        x = self.initial_bn2(x)
        x = self.initial_relu2(x)

        x = self.layer1(x)

        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


class VGG_Q2(nn.Module):
    def __init__(self, ):
        super(VGG_Q2, self).__init__()
        self.init_channels = 64
        self.num_classes = 10
        self.features = self._make_layers()
        self.classifier = nn.Sequential(
            nn.Linear(13, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, self.num_classes),
        )


    def _make_layers(self):
        layers = []
        in_channels = 3
        current_channels = self.init_channels
        kernel_size = 3

        cfg = [2, 2, 3, 3, 3]

        for num_convs in cfg:
            for _ in range(num_convs):
                layers.append(nn.Conv2d(in_channels, current_channels, kernel_size=kernel_size, padding=kernel_size//2))
                layers.append(nn.BatchNorm2d(current_channels))
                layers.append(nn.ReLU(inplace=True))
                in_channels = current_channels


            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

            # Reduce channels by 35%
            current_channels = math.ceil(current_channels * 0.65)

            # Increase kernel size by 25% with ceil rounding
            kernel_size = math.ceil(kernel_size * 1.25)

            if kernel_size % 2 == 0:  # Ensure kernel size is odd for padding consistency
                kernel_size += 1

        return nn.Sequential(*layers)

    def forward(self,x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x



class CNABlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(CNABlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self,x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

class InceptionBlock(nn.Module):
    def __init__(self, in_channels, final_out_channels): # DO NOT CHANGE __init__ arguements
        super().__init__()

        self.branch1x1 = CNABlock(in_channels, 64, kernel_size=1)

        self.branch5x5_1 = CNABlock(in_channels, 48, kernel_size=3, padding=1)
        self.branch5x5_2 = CNABlock(48, 64, kernel_size=5, padding=2)

        self.branch5x5dbl_1 = CNABlock(in_channels, 48, kernel_size=3, padding=1)
        self.branch5x5dbl_2 = CNABlock(48, 64, kernel_size=5, padding=2)

        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            CNABlock(in_channels, 32, kernel_size=1)
        )

        # Compute the number of output channels after concatenation
        concat_out_channels = 64 + 64 + 64 + 32
        self.filter = CNABlock(concat_out_channels, final_out_channels, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch5x5dbl = self.branch5x5dbl_1(x)
        branch5x5dbl = self.branch5x5dbl_2(branch5x5dbl)

        branch_pool = self.branch_pool(x)

        outputs = [branch1x1, branch5x5, branch5x5dbl, branch_pool]
        x = torch.cat(outputs, 1)
        x = self.filter(x)
        return x

class Inception_Q3(nn.Module):
    def __init__(self):
        super(Inception_Q3, self).__init__()
        self.stem = nn.Sequential(
            CNABlock(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.inception1 = InceptionBlock(in_channels=64, final_out_channels=128)
        self.inception2 = InceptionBlock(in_channels=128, final_out_channels=256)
        self.inception3 = InceptionBlock(in_channels=256, final_out_channels=512)
        self.inception4 = InceptionBlock(in_channels=512, final_out_channels=1024)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, 10)

    def forward(self,x):
        x = self.stem(x)
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.inception3(x)
        x = self.inception4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

class CustomNetwork_Q4(nn.Module):
    def __init__(self,) :
        super(CustomNetwork_Q4, self).__init__()
        self.stem = nn.Sequential(
            CNABlock(in_channels=3, out_channels=128, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.residual1 = ResidualBlock(in_channels=128, out_channels=128)
        self.residual2 = ResidualBlock(in_channels=128, out_channels=128)
        self.inception1 = InceptionBlock(in_channels=128, final_out_channels=256)
        self.inception2 = InceptionBlock(in_channels=256, final_out_channels=512)
        self.residual3 = ResidualBlock(in_channels=512, out_channels=512)
        self.inception3 = InceptionBlock(in_channels=512, final_out_channels=1024)
        self.residual4 = ResidualBlock(in_channels=1024, out_channels=1024)
        self.inception4 = InceptionBlock(in_channels=1024, final_out_channels=2048)
        self.residual5 = ResidualBlock(in_channels=2048, out_channels=2048)
        self.inception5 = InceptionBlock(in_channels=2048, final_out_channels=2048)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, 10)

    def forward(self,x):
        x = self.stem(x)
        x = self.residual1(x)
        x = self.residual2(x)
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.residual3(x)
        x = self.inception3(x)
        x = self.residual4(x)
        x = self.inception4(x)
        x = self.residual5(x)
        x = self.inception5(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x



def trainer(gpu="F",
            dataloader=None,
            network=None,
            criterion=None,
            optimizer=None,
            epoch=None):
    device = torch.device("cuda" if torch.cuda.is_available() and gpu == "T" else "cpu")
    network.to(device)
    #print(list(network.parameters()))

    network.train()
    #running_loss = 0.0
    prev_loss = 10000
    count = 0

    for i in range(0,epoch):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = network(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            #print(list(network.parameters()))

            running_loss += loss.item()

        avg_loss = running_loss / len(dataloader)
        if prev_loss - avg_loss < 0.0000000001:
            count += 1
        else:
            count = 0
        if count > 50:
            break
        prev_loss = avg_loss
        if i%5 == 0:
            print(f"Training Loss: {avg_loss:.4f} at epoch: {i+1}")


def validator(gpu="F",
              dataloader=None,
              network=None,
              criterion=None,
              optimizer=None):
    device = torch.device("cuda" if torch.cuda.is_available() and gpu == "T" else "cpu")
    network.to(device)

    network.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = network(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(dataloader)
    accuracy = 100 * correct / total
    print(f"Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


def test(gpu="F",
              dataloader=None,
              network=None,
              criterion=None,
              optimizer=None):

    device = torch.device("cuda" if torch.cuda.is_available() and gpu == "T" else "cpu")
    network.to(device)

    network.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = network(inputs)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")


100%|██████████| 170498071/170498071 [00:13<00:00, 13069976.67it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:

BATCH_SIZE = 128
LEARNING_RATE = 0.0001
EPOCH = 100
GPU ='T'

if __name__ == "__main__":

    imageDataset = [
        ImageDataset(split="train"),
        ImageDataset(split="val"),
        ImageDataset(split="test")

    ]



    Architectures = [
        Resnet_Q1(),
        VGG_Q2(),
        Inception_Q3(),
        CustomNetwork_Q4()
    ]


    for network in Architectures:

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(
            params=network.parameters(),
            lr=LEARNING_RATE
        )

        for dataset in imageDataset:
            if dataset.datasplit == "train":
                print(
                    "Training {} Architecture on {} split of {}".format(
                        network.__class__.__name__,
                        dataset.datasplit,
                        dataset.__class__.__name__
                    )
                )
                network.train()
                train_dataloader = DataLoader(
                    dataset=dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=2,
                    drop_last=True
                )

                trainer(
                    gpu=GPU,
                    dataloader=train_dataloader,
                    network=network,
                    criterion=criterion,
                    optimizer=optimizer,
                    epoch=EPOCH
                )

            elif dataset.datasplit == "val":
                print(
                    "Validating {} Architecture on {} split of {}".format(
                        network.__class__.__name__,
                        dataset.datasplit,
                        dataset.__class__.__name__
                    )
                )
                network.train()
                val_dataloader = DataLoader(
                    dataset=dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=2,
                    drop_last=True
                )

                validator(
                    gpu=GPU,
                    dataloader=val_dataloader,
                    network=network,
                    criterion=criterion,
                    optimizer=optimizer
                )

            elif dataset.datasplit == "test":
                print(
                    "Testing {} Architecture on {} split of {}".format(
                        network.__class__.__name__,
                        dataset.datasplit,
                        dataset.__class__.__name__
                    )
                )
                network.eval()
                test_dataloader = DataLoader(
                    dataset=dataset,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=2,
                    drop_last=True
                )
                test(
                    gpu=GPU,
                    dataloader=test_dataloader,
                    network=network,
                    criterion=None,
                    optimizer=None)


Training Resnet_Q1 Architecture on train split of ImageDataset
Training Loss: 2.6775 at epoch: 1
Training Loss: 0.5471 at epoch: 6
Training Loss: 0.1258 at epoch: 11
Training Loss: 0.0966 at epoch: 16
Training Loss: 0.0746 at epoch: 21
Training Loss: 0.0795 at epoch: 26
Training Loss: 0.0352 at epoch: 31
Training Loss: 0.0538 at epoch: 36
Training Loss: 0.0322 at epoch: 41
Training Loss: 0.0405 at epoch: 46
Training Loss: 0.0241 at epoch: 51
Training Loss: 0.0280 at epoch: 56
Training Loss: 0.0261 at epoch: 61
Training Loss: 0.0165 at epoch: 66
Training Loss: 0.0092 at epoch: 71
Training Loss: 0.0215 at epoch: 76
Training Loss: 0.0100 at epoch: 81
Training Loss: 0.0102 at epoch: 86
Training Loss: 0.0095 at epoch: 91
Training Loss: 0.0136 at epoch: 96
Validating Resnet_Q1 Architecture on val split of ImageDataset
Validation Loss: 2.5647, Accuracy: 68.51%
Testing Resnet_Q1 Architecture on test split of ImageDataset
Test Accuracy: 67.92%
Training VGG_Q2 Architecture on train split of Imag